In [ ]:
from pathlib import Path

In [ ]:
image_dir = Path('../tests/data/mixed_images')

# Use one of the prepackaged models

In [ ]:
from imagededup.methods import CNN

# Get CustomModel construct
from imagededup.utils import CustomModel

# Get efficientnet model
from imagededup.utils.models import EfficientNet
# Other models include ViT, MobilenetV3 (default selection)

In [ ]:
# Declare a custom config with CustomModel, the prepackaged models come with a name and transform function
custom_config = CustomModel(name=EfficientNet.name,
                            model=EfficientNet(),
                            transform=EfficientNet.transform)

In [ ]:
cnn_encoder = CNN(model_config=custom_config)

In [ ]:
duplicates_cnn = cnn_encoder.find_duplicates(image_dir=image_dir, scores=True)

In [ ]:
duplicates_cnn

# User-defined model

In [ ]:
from imagededup.methods import CNN

# Get CustomModel construct
from imagededup.utils import CustomModel

# Import necessary pytorch constructs for initializing a custom feature extractor
import torch
from torchvision.transforms import transforms

# Declare custom feature extractor class
class MyModel(torch.nn.Module):
    transform = transforms.Compose(
        [
            transforms.Resize((256, 256)),
            transforms.ToTensor()
        ]
    ) # transform must take PIL.Image as input and return a torch.Tensor

    name = 'my_custom_model' # name can be any user-defined string

    def __init__(self):
        super().__init__()
        # Define the layers of the model here

    def forward(self, x):
        # Add more operations here
        x = x.view(-1, 256*256*3) # output shape: batch_size x features
        return x

In [ ]:
# Initialize the CNN using model_config parameter and setting it to the custom model
custom_config = CustomModel(name=MyModel.name,
                            model=MyModel(),
                            transform=MyModel.transform)

cnn = CNN(model_config=custom_config)

In [ ]:
duplicates_cnn = cnn.find_duplicates(image_dir=image_dir, scores=True)

In [ ]:
duplicates_cnn

# Using a huggingface model

In [ ]:
!pip install transformers

In [ ]:
from imagededup.methods import CNN

# Get CustomModel construct
from imagededup.utils import CustomModel

# Import necessary constructs for initializing a huggingface transformers model
from transformers import ViTModel, AutoImageProcessor
import torch
from torchvision.transforms import transforms

VIT_MODEL = "google/vit-base-patch16-224-in21k"

def vit_transform(image):
    transform = AutoImageProcessor.from_pretrained(VIT_MODEL)
    x = transform(image, return_tensors = 'pt')['pixel_values']
    return x

class VitHgface(torch.nn.Module):
    transform = transforms.Lambda(vit_transform)

    name = 'ViT_hgface'

    def __init__(self):
        super().__init__()
        self.vit = ViTModel.from_pretrained(VIT_MODEL)

    def forward(self, x):
        x  = x.view(-1, 3, 224, 224)
        with torch.no_grad():
            out = self.vit(pixel_values=x)
        return out.pooler_output

In [ ]:
custom_config = CustomModel(name=VitHgface.name,
                            model=VitHgface(),
                            transform=VitHgface.transform)

In [ ]:
cnn = CNN(model_config=custom_config)

In [ ]:
duplicates_cnn = cnn.find_duplicates(image_dir=image_dir, scores=True)

In [ ]:
duplicates_cnn

In [ ]:
enc = cnn.encode_images(image_dir=image_dir)